<a href="https://colab.research.google.com/github/rsemihkoca/YouDo-ds-bc/blob/main/Implicit_TY_Grad_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implicit Library Implementation

In [1]:
!pip install --upgrade implicit

     |████████████████████████████████| 18.5 MB 3.8 MB/s 


In [2]:
#Implicit warning 
import os; os.environ['OPENBLAS_NUM_THREADS']='1'

import numpy as np
import pandas as pd
import implicit
from scipy.sparse import csr_matrix
from implicit.evaluation import mean_average_precision_at_k

# Load dataframes

In [3]:
%%time
from google.colab import output
!pip install cache_magic
output.clear()

from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab_Notebooks/'
output.clear()

import cache_magic
!ln -s '/content/drive/MyDrive/Colab_Notebooks/.cache' /content/.cache
output.clear()

%cache transactions_df = pd.read_csv("https://storage.googleapis.com/gm-datascience-share/capstone/transactions.csv.gz", dtype={'article_id': str}, parse_dates=['t_dat'])
#df_sub = pd.read_csv(csv_sub)
%cache customers_df = pd.read_csv("https://storage.googleapis.com/gm-datascience-share/capstone/customers.csv.gz")
%cache items_df = pd.read_csv("https://storage.googleapis.com/gm-datascience-share/capstone/articles.csv.gz", dtype={'article_id': str})

Loading cached value for variable 'transactions_df'. Time since caching: 3:15:13.998902
Loading cached value for variable 'customers_df'. Time since caching: 3:14:58.811217
Loading cached value for variable 'items_df'. Time since caching: 3:14:57.880198
CPU times: user 13.7 s, sys: 3.08 s, total: 16.8 s
Wall time: 1min 3s


In [4]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       object        
 2   article_id        object        
 3   price             float64       
 4   sales_channel_id  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 1.2+ GB


In [5]:
# Veriyi küçülterek hızlı çıktı almaya çalıştım
transactions_df = transactions_df[transactions_df['t_dat'] > '2019-09-15']
transactions_df.shape

(15130701, 5)

In [6]:
(transactions_df['t_dat'].max()-transactions_df['t_dat'].min()).days

372

## Memory Optimization with Incremental Indexing

In [7]:
ALL_USERS = customers_df['customer_id'].unique().tolist()
ALL_ITEMS = items_df['article_id'].unique().tolist()

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

transactions_df['user_id'] = transactions_df['customer_id'].map(user_map)
transactions_df['item_id'] = transactions_df['article_id'].map(item_map)

del customers_df, items_df

In [8]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15130701 entries, 16657623 to 31788323
Data columns (total 7 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       object        
 2   article_id        object        
 3   price             float64       
 4   sales_channel_id  int64         
 5   user_id           int64         
 6   item_id           int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 923.5+ MB


In [9]:
transactions_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id,user_id,item_id
16657623,2019-09-16,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,708098001,0.020322,2,47,54561
16657624,2019-09-16,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,640021011,0.050831,2,47,33867
16657625,2019-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,652730006,0.050831,2,330,37111
16657626,2019-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,652730006,0.050831,2,330,37111
16657627,2019-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,516859001,0.013542,2,330,7722


# CSR Matrix for Sparse Rating Matrix

In [10]:
row = transactions_df['user_id'].values
col = transactions_df['item_id'].values
data = np.ones(transactions_df.shape[0]) 
csr_train = csr_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
csr_train

<1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
	with 13075307 stored elements in Compressed Sparse Row format>

# Do Implicit support CSR Matrix ?

In [11]:
%%time
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=2)
model.fit(csr_train)

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 1.28 s, sys: 258 ms, total: 1.54 s
Wall time: 2.71 s


# Yes, it doess 🏄‍♂️ :D

# Train-Test-Validation Split and Model Creation

In [12]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [13]:
implicit.gpu.HAS_CUDA

True

In [27]:
# from sklearn.model_selection import train_test_split

# df_train,df_test = train_test_split(coo_train, test_size=0.2, random_state=42)
# df_train

<1097584x105542 sparse matrix of type '<class 'numpy.float64'>'
	with 840530 stored elements in Compressed Sparse Row format>

In [ ]:
-**time sensitivity
+**2018 de yaptığı seçimi değersizleştirmem gerekiyor**
+**Sezonsallığı kurtarmak için veriyi 1 sene olarak büyütmeye çalış bunun için cython implicit kullandım sezonsallık bilgisini almak için**
**implicit 0 alması begenmedigi anlamına gelmiyor haberi yok bu yüzden confidence interval ve time decay ekle**
-**Aynı ürünü tekrar alma neden açık yaz**
+**implicit set true unutma**
+**conf interval

## Functions required for validation

In [36]:
def TO_User_Item_CSRMatrix(df):
    #Turn a dataframe with transactions into a CSR sparse items x users matrix

    row = df['user_id'].values
    col = df['item_id'].values
    data = np.ones(df.shape[0])
    csr = csr_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
    return csr

In [37]:
def Train_Test_Validation_Split(df, validation_days=7,Test=True):
  if Test:
      from sklearn.model_selection import train_test_split

      validation_cut = df['t_dat'].max() - pd.Timedelta(validation_days)
      #Get start date of validation 

      df_rest = df[df['t_dat'] < validation_cut]

      #Create validation week(Last week in transactions)
      df_val = df[df['t_dat'] >= validation_cut]

      df_train,df_test = train_test_split(df_rest, test_size=0.2, random_state=42)

      return df_train, df_val, df_test
      
  elif Test==False:
      validation_cut = df['t_dat'].max() - pd.Timedelta(validation_days)
      #Get start date of validation 

      df_train = df[df['t_dat'] < validation_cut]

      #Create validation week(Last week in transactions)
      df_val = df[df['t_dat'] >= validation_cut]

      return df_train,df_val

In [38]:
def Get_all_matrices_with_conf(df, validation_days=7,Test=True):
  if Test:
    df_train, df_val, df_test = Train_Test_Validation_Split(df, validation_days=validation_days,Test=True)

    csr_train = TO_User_Item_CSRMatrix(df_train)
    csr_val = TO_User_Item_CSRMatrix(df_val)
    csr_test = TO_User_Item_CSRMatrix(df_test)
    
    alpha_val = 40
    csr_train = (csr_train * alpha_val).astype('double')  
    csr_val = (csr_val * alpha_val).astype('double')  
    csr_test = (csr_test * alpha_val).astype('double')  
    return {
            'csr_train': csr_train,
            'csr_val': csr_val,
            'csr_test': csr_test
          }
  else:
    df_train, df_val = Train_Test_Validation_Split(df, validation_days=validation_days,Test=False)

    csr_train = TO_User_Item_CSRMatrix(df_train)
    csr_val = TO_User_Item_CSRMatrix(df_val)
    
    alpha_val = 40
    csr_train = (csr_train * alpha_val).astype('double')  
    csr_val = (csr_val * alpha_val).astype('double')  
    return {
            'csr_train': csr_train,
            'csr_val': csr_val,
          }


In [18]:
def validation(matrices, factors=200, iterations=20, regularization=0.01, show_progress=True):

    csr_train, csr_val, csr_test = matrices['csr_train'], matrices['csr_val'], matrices['csr_test']

    model = implicit.als.AlternatingLeastSquares(factors=factors, #hyperparameter1
                                                 regularization=regularization, #hyperparameter2
                                                 use_native=True,
                                                 use_cg=True,
                                                 use_gpu=True,
                                                 iterations=iterations, #hyperparameter3
                                                 calculate_training_loss=True,
                                                 random_state=42)

    model.fit(csr_train, show_progress=show_progress)

    # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
    # TODO: change MAP@12 to a library that allows repeated items in prediction
    map12 = mean_average_precision_at_k(model, csr_train, csr_test, K=12, show_progress=False, num_threads=4)
    print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} ==> MAP@12: {map12:6.5f}")
    return map12

In [20]:
X=transactions_df[["t_dat","user_id","item_id"]]


In [21]:
matrices = Get_all_matrices_with_conf(X, validation_days=7,Test=True)

In [22]:
%%time
best_map12 = 0
for factors in [200, 500, 750, 1000]:
    for iterations in [3,20,45]:
        for regularization in [0.01]:
            print("******************")
            map12 = validation(matrices, factors, iterations, regularization, show_progress=True)
            if map12 > best_map12:
                best_map12 = map12
                best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization}
                print(f"Best MAP@12 found. Updating: {best_params}")

  0%|          | 0/3 [00:00<?, ?it/s]

Factors: 200 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.03375
Best MAP@12 found. Updating: {'factors': 200, 'iterations': 3, 'regularization': 0.01}


  0%|          | 0/20 [00:00<?, ?it/s]

Factors: 200 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.03923
Best MAP@12 found. Updating: {'factors': 200, 'iterations': 20, 'regularization': 0.01}


  0%|          | 0/45 [00:00<?, ?it/s]

...

Factors: 1000 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.04703


  0%|          | 0/20 [00:00<?, ?it/s]

Factors: 1000 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.05583
Best MAP@12 found. Updating: {'factors': 1000, 'iterations': 20, 'regularization': 0.01}


  0%|          | 0/45 [00:00<?, ?it/s]

Factors: 1000 - Iterations: 45 - Regularization: 0.010 ==> MAP@12: 0.05588
Best MAP@12 found. Updating: {'factors': 1000, 'iterations': 45, 'regularization': 0.01}
CPU times: user 56min 12s, sys: 7.89 s, total: 56min 20s
Wall time: 49min 49s


In [ ]:
#del matrices
Best MAP@12 found. Updating: {'factors': 1000, 'iterations': 45, 'regularization': 0.01}


In [ ]:
# 100%
# 3/3 [00:04<00:00, 1.32s/it, loss=-.311]
# Factors: 200 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.03375
# Best MAP@12 found. Updating: {'factors': 200, 'iterations': 3, 'regularization': 0.01}
# 100%
# 20/20 [00:25<00:00, 1.26s/it, loss=-.284]
# Factors: 200 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.03923
# Best MAP@12 found. Updating: {'factors': 200, 'iterations': 20, 'regularization': 0.01}
# 100%
# 45/45 [00:57<00:00, 1.27s/it, loss=-.283]
# Factors: 200 - Iterations: 45 - Regularization: 0.010 ==> MAP@12: 0.03860
# 100%
# 3/3 [00:15<00:00, 4.83s/it, loss=-.236]
# Factors: 500 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.04345
# Best MAP@12 found. Updating: {'factors': 500, 'iterations': 3, 'regularization': 0.01}
# 100%
# 20/20 [01:43<00:00, 5.16s/it, loss=-.206]
# Factors: 500 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.04970
# Best MAP@12 found. Updating: {'factors': 500, 'iterations': 20, 'regularization': 0.01}
# 100%
# 45/45 [03:52<00:00, 5.14s/it, loss=-.202]
# Factors: 500 - Iterations: 45 - Regularization: 0.010 ==> MAP@12: 0.04920
# 100%
# 3/3 [00:32<00:00, 10.19s/it, loss=-.207]
# Factors: 750 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.04608
# 100%
# 20/20 [03:36<00:00, 10.84s/it, loss=-.17]
# Factors: 750 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.05374
# Best MAP@12 found. Updating: {'factors': 750, 'iterations': 20, 'regularization': 0.01}
# 100%
# 45/45 [08:08<00:00, 10.85s/it, loss=-.164]
# Factors: 750 - Iterations: 45 - Regularization: 0.010 ==> MAP@12: 0.05348
# 100%
# 3/3 [01:11<00:00, 22.32s/it, loss=-.186]
# Factors: 1000 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.04703
# 100%
# 20/20 [07:53<00:00, 23.67s/it, loss=-.144]
# Factors: 1000 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.05583
# Best MAP@12 found. Updating: {'factors': 1000, 'iterations': 20, 'regularization': 0.01}
# 100%
# 45/45 [17:45<00:00, 23.68s/it, loss=-.138]
# Factors: 1000 - Iterations: 45 - Regularization: 0.010 ==> MAP@12: 0.05588
# Best MAP@12 found. Updating: {'factors': 1000, 'iterations': 45, 'regularization': 0.01}
# CPU times: user 56min 12s, sys: 7.89 s, total: 56min 20s
# Wall time: 49min 49s

# Training over the full dataset

In [45]:
matrices=Get_all_matrices_with_conf(X, validation_days=7,Test=False)


In [44]:
def train(matrices, factors=200, iterations=20, regularization=0.01, show_progress=True):

    csr_train, csr_val = matrices['csr_train'], matrices['csr_val']

    model = implicit.als.AlternatingLeastSquares(factors=factors, #hyperparameter1
                                                 regularization=regularization, #hyperparameter2
                                                 use_native=True,
                                                 use_cg=True,
                                                 use_gpu=True,
                                                 iterations=iterations, #hyperparameter3
                                                 calculate_training_loss=True,
                                                 random_state=42)

    model.fit(csr_train, show_progress=show_progress)
    map12 = mean_average_precision_at_k(model, csr_train, csr_val, K=12, show_progress=False, num_threads=4)
    print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} ==> MAP@12: {map12:6.5f}")
    return model

 

In [43]:
best_params

{'factors': 1000, 'iterations': 45, 'regularization': 0.01}

In [46]:
model = train(matrices, **best_params)

  0%|          | 0/45 [00:00<?, ?it/s]

Factors: 1000 - Iterations: 45 - Regularization: 0.010 ==> MAP@12: 0.00580


# Recommendation-Prediction

In [50]:
def submit(model, csr_train, result_file="recommendations.csv"):
    preds = []
    batch_size = 2000
    to_generate = np.arange(len(ALL_USERS))
    for startidx in range(0, len(to_generate), batch_size):
        batch = to_generate[startidx : startidx + batch_size]
        ids, scores = model.recommend(batch, csr_train[batch], N=12, filter_already_liked_items=False)
        for i, userid in enumerate(batch):
            customer_id = user_ids[userid]
            user_items = ids[i]
            article_ids = [item_ids[item_id] for item_id in user_items]
            preds.append((customer_id, ' '.join(article_ids)))

    df_preds = pd.DataFrame(preds, columns=['customer_id', 'prediction'])
    df_preds.to_csv(result_file, index=False)
    
    display(df_preds.head())
    print(df_preds.shape)
    
    return df_preds

In [51]:
%%time
df_preds = submit(model, matrices['csr_train']);

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,795440001 785710001 797065001 841260003 812683...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,590928022 739590027 811925005 599580055 559630...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,351484002 723529001 870304002 750424014 851400...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,187949032 187949031 187949030 187949029 187949...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,730683050 896152002 827971001 791587015 818320...


(1371980, 2)
CPU times: user 1min 24s, sys: 874 ms, total: 1min 25s
Wall time: 1min 24s
